# Ion channel challenge: features generated per batch, with improved computation of square root

First, we generated features on the whole set. This method failed to acknowledge differences between batchs, and the rolling statistics windows of different batches were overlapping ([notebook 1](https://github.com/bd3thier/Ion-channels/blob/master/notebooks/Ion%20channel%20-%20Feature%20engineering%201.ipynb)).

Then, the features were generated per batch ([notebook 2](https://github.com/bd3thier/Ion-channels/blob/master/notebooks/Ion%20channel%20-%20Feature%20engineering%202.ipynb)).

In this notebook, we also include an improved way to compute the square root of the signal so negative values are not returning NAs.

In [1]:
# Load packages
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
import seaborn as sns
from sklearn import metrics
from sklearn.metrics import f1_score
import torch
from tqdm import tqdm
from sklearn.model_selection import train_test_split


In [2]:
# from https://www.kaggle.com/kmat2019/u-net-1d-cnn-with-keras
def reduce_mem_usage(df: pd.DataFrame,
                     verbose: bool = True) -> pd.DataFrame:
    
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2

    for col in df.columns:
        col_type = df[col].dtypes

        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()

            if str(col_type)[:3] == 'int':

                if (c_min > np.iinfo(np.int32).min
                      and c_max < np.iinfo(np.int32).max):
                    df[col] = df[col].astype(np.int32)
                elif (c_min > np.iinfo(np.int64).min
                      and c_max < np.iinfo(np.int64).max):
                    df[col] = df[col].astype(np.int64)
            else:
                if (c_min > np.finfo(np.float16).min
                        and c_max < np.finfo(np.float16).max):
                    df[col] = df[col].astype(np.float16)
                elif (c_min > np.finfo(np.float32).min
                      and c_max < np.finfo(np.float32).max):
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    reduction = (start_mem - end_mem) / start_mem

    msg = f'Mem. usage decreased to {end_mem:5.2f} MB ({reduction * 100:.1f} % reduction)'
    if verbose:
        print(msg)

    return df

In [3]:
# for each window in the list, calculate the aggregation function twice (
#the second time the winow is centered, not to the left of value being computed)
# for each window in the list, calculate the aggregation function twice (
#the second time the winow is centered, not to the left of value being computed)
def feature_engineering(df, windows=[5, 15, 45, 135, 405, 1215], i=0, signal_size = 500000):
    '''Create new features from 'signal' (rolling statistics , lag, power and gradient) and concatenate them 
    to the existing dataframe'''
    for window in tqdm(windows):
        # Basic aggregation functions
        # mean 
        name = 'mean_'+str(window)+'_r'
        df[name] = df.signal.rolling(window = window, min_periods = 1).mean()
        name = 'mean_'+str(window)+'_c'
        df[name] = df.signal.rolling(window = window, center = True, min_periods = 1).mean()
        # standard dev
        name = 'std_'+str(window)+'_r'
        df[name] = df.signal.rolling(window = window, min_periods = 1).std()
        name = 'std_'+str(window)+'_c'
        df[name] = df.signal.rolling(window = window, center = True, min_periods = 1).std()
        # min
        name = 'min_'+str(window)+'_r'
        df[name] = df.signal.rolling(window = window, min_periods = 1).min()
        name = 'min_'+str(window)+'_c'
        df[name] = df.signal.rolling(window = window, center = True, min_periods = 1).min()
        # max
        name = 'max_'+str(window)+'_r'
        df[name] = df.signal.rolling(window = window, min_periods = 1).max()
        name = 'max_'+str(window)+'_c'
        df[name] = df.signal.rolling(window = window, center = True, min_periods = 1).max()
        # skew 
        name = 'skew_'+str(window)+'_r'
        df[name] = df.signal.rolling(window = window, min_periods = 1).skew()
        name = 'skew_'+str(window)+'_c'
        df[name] = df.signal.rolling(window = window, center = True, min_periods = 1).skew() 
        # kurtosis 
        name = 'kurt_'+str(window)+'_r'
        df[name] = df.signal.rolling(window = window, min_periods = 1).kurt()
        name = 'kurt_'+str(window)+'_c'
        df[name] = df.signal.rolling(window = window, center = True, min_periods = 1).kurt()     
        
        # Exponentially weighted functions (values closed to value being computed have more weight)
        # mean 
        name = 'mean_'+str(window)+'_r_ew'
        df[name] = df.signal.rolling(window = window, min_periods = 1, win_type='exponential').mean(tau = 1)
        name = 'mean_'+str(window)+'_c_ew'
        df[name] = df.signal.rolling(window = window, center = True, min_periods = 1, win_type='exponential').mean(tau = 1)
        # standard dev
        #name = 'std_'+str(window)+'_r_ew'
        #df[name] = df.signal.rolling(window = window, min_periods = 1, win_type='exponential').std(tau = 1)
        #name = 'std_'+str(window)+'_c_ew'
        #df[name] = df.signal.rolling(window = window, center = True, min_periods = 1, win_type='exponential').std(tau = 1)
        
        # Span max/min
        name = 'max_min_diff_'+str(window) + '_r'
        df[name] = df['max_'+str(window)+'_r'] - df['min_'+str(window)+'_r']
        name = 'min_max_ratio_'+str(window) + '_r'
        df[name] = (df['max_'+str(window)+'_r'] - df['min_'+str(window)+'_r'])/df.signal
        name = 'min_max_ratio_mean_'+str(window) + '_r'
        df[name] = (df['max_'+str(window)+'_r'] - df['min_'+str(window)+'_r'])/df.signal.rolling(window = window).mean()
        name = 'max_min_diff_'+str(window) + '_c'
        df[name] = df['max_'+str(window)+'_c'] - df['min_'+str(window)+'_c']
        name = 'min_max_ratio_'+str(window) + '_c'
        df[name] = (df['max_'+str(window)+'_c'] - df['min_'+str(window)+'_c'])/df.signal
        name = 'min_max_ratio_mean_'+str(window) + '_c'
        df[name] = (df['max_'+str(window)+'_c'] - df['min_'+str(window)+'_c'])/df.signal.rolling(window = window).mean()
    
    
    df['lag-1'] = df.signal.shift(1, fill_value=df.signal.mean())
    df['lag-3'] = df.signal.shift(3)
    df['lag-5'] = df.signal.shift(5)
    df['lag+1'] = df.signal.shift(-1)
    df['lag+2'] = df.signal.shift(-2)
    df['power2'] = df.signal**2
    df['sqroot'] = np.sqrt(np.abs(df.signal)) * np.sign(df.signal)
    df['power3'] = df.signal**3 
    df['deriv'] = np.gradient(df.signal)
    df['deriv_lag-1'] = df.deriv.shift(1) 
    df['deriv_lag-3'] =  df.deriv.shift(3) 
    df['deriv_lag+1'] =  df.deriv.shift(-1)
    df['deriv_lag+3'] =  df.deriv.shift(-3)
    df['exp'] = np.exp(df.signal)
 
    return df
        

In [4]:
# For the integral, we need a normalized signal, however it only makes sense to do it per batch
def normalize_per_batch(df, signal_size=500000):
    df['signal_norm'] = 0
    for i in tqdm(range(int(df.shape[0]/signal_size))):
        # normalize
        mean = df.signal[i*signal_size:(i+1)*signal_size].mean()
        df.signal_norm[i*signal_size:(i+1)*signal_size] = df.signal[i*signal_size:(i+1)*signal_size] - mean
    return df

# Generate integration and integration over last 10 timesteps
def feature_engineering_integral(df, signal_size=500000):
    #df['signal_norm'] = 0
    for i in tqdm(range(int(df.shape[0]/signal_size))):
        # normalize
        mean = df.signal[i*signal_size:(i+1)*signal_size].mean()
        df.loc[i*signal_size:(i+1)*signal_size, 'signal_norm'] = df.signal[i*signal_size:(i+1)*signal_size] - mean
    df['integration'] = np.cumsum(df.signal_norm)

    df['integration_shift10'] = (df['integration'] -df['integration'].shift(10))
    # We need to remove the normalized signal column in the end
    df.drop('signal_norm', axis = 1, inplace = True)
    return df
        

In [5]:
# Make sure we have no NAs or infinite values
def feature_cleaning(train, test):
    features = [col for col in train.columns if col not in ['index', 'group', 'open_channels', 'time']]
    train = train.replace([np.inf, -np.inf], np.nan)
    test = test.replace([np.inf, -np.inf], np.nan)
    for feature in tqdm(features):
        feature_mean = train[feature].mean()
        train[feature] = train[feature].fillna(feature_mean)
        test[feature] = test[feature].fillna(feature_mean)
    return train, test, features

In [6]:
# from https://www.kaggle.com/martxelo/fe-and-ensemble-mlp-and-lgbm
# but I want to normalize per batch

def divide_and_add_features(df, signal_size=500000, windows =[5, 15, 45, 135, 405, 1215] ):
    
    new_df = pd.DataFrame()
    for i in tqdm(range(int(df.shape[0]/signal_size))):
        new_features_one_batch = feature_engineering(df[i*signal_size:(i+1)*signal_size], windows, i, signal_size)
        new_df = pd.concat([new_df, new_features_one_batch], axis=0)
    
    #feature_engineering_integral(df, signal_size)
    
    return new_df


The signal with kalman filter was downloaded from [michaln](https://www.kaggle.com/michaln/data-without-drift-with-kalman-filter)

In [7]:
# Load data
train = pd.read_csv('../data/external/train_kalman.csv')
test = pd.read_csv('../data/external/test_kalman.csv')

## Generating features

In this version there are no NA values at the edges of rolling statistics thanks to the min_periods = 1 argument

In [8]:
windows = [5, 15, 45, 135, 405, 1215]

In [9]:
def test_harness_before(train = train, test = test, windows = windows, keep_na = True):   
    train = divide_and_add_features(train)
    train = feature_engineering_integral(train)
    train = reduce_mem_usage(train)
    
    test = divide_and_add_features(test)
    test = feature_engineering_integral(test)

    #test = reduce_mem_usage(test)
    # removed because it changes the format of the 'time' feature (rounds to 2 decimal places), resulting in a 
    # format error upon submission
    
    if keep_na == False:
        train, test, features = feature_cleaning(train, test)
    
    y = train.open_channels
    X = train.drop('open_channels', axis=1)
    X_train, X_validation, y_train, y_validation = train_test_split(X, y, train_size=0.8, random_state=134)
    return train, test, X_train, X_validation, y_train, y_validation

In [10]:
train, test, X_train, X_validation, y_train, y_validation = test_harness_before()


  0%|          | 0/6 [00:00<?, ?it/s]C:\Users\beren\anaconda3\envs\TensorFlow-GPU\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
C:\Users\beren\anaconda3\envs\TensorFlow-GPU\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\beren\anaconda3\envs\TensorFlow-GPU\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try

  0%|          | 0/6 [00:00<?, ?it/s]

Mem. usage decreased to 1792.91 MB (66.2 % reduction)



100%|██████████| 4/4 [00:01<00:00,  2.31it/s]


In [11]:
train.shape

(5000000, 139)

In [12]:
train.tail(20)

,time,signal,open_channels,mean_5_r,mean_5_c,std_5_r,std_5_c,min_5_r,min_5_c,max_5_r,...,sqroot,power3,deriv,deriv_lag-1,deriv_lag-3,deriv_lag+1,deriv_lag+3,exp,integration,integration_shift10
4999980,500.0,4.203125,8,4.648438,4.117188,1.024414,0.393555,3.443359,3.443359,5.980469,...,2.048828,74.125000,0.410156,-0.001000,-0.601562,0.138306,0.540527,66.750000,-17.406250,19.125000
4999981,500.0,4.265625,8,4.417969,4.054688,0.936035,0.401123,3.443359,3.443359,5.980469,...,2.064453,77.562500,0.138306,0.410156,-1.267578,-0.186523,0.694824,71.125000,-16.453125,18.328125
4999982,500.0,4.476562,8,4.117188,4.480469,0.393555,0.639160,3.443359,3.890625,4.476562,...,2.115234,89.812500,-0.186523,0.138306,-0.001000,0.540527,-0.225464,88.062500,-15.289062,16.031250
4999983,500.0,3.890625,8,4.054688,4.695312,0.401123,0.701172,3.443359,3.890625,4.476562,...,1.972656,58.937500,0.540527,-0.186523,0.410156,0.694824,-0.480713,48.968750,-14.710938,13.554688
4999984,500.0,5.558594,9,4.480469,4.863281,0.639160,0.672852,3.890625,3.890625,5.558594,...,2.357422,171.750000,0.694824,0.540527,0.138306,-0.225464,-0.634277,259.500000,-12.460938,13.476562
4999985,500.0,5.281250,9,4.695312,4.832031,0.701172,0.698730,3.890625,3.890625,5.558594,...,2.298828,147.250000,-0.225464,0.694824,-0.186523,-0.480713,-0.343262,196.500000,-10.492188,13.601562
4999986,500.0,5.109375,9,4.863281,4.820312,0.672852,0.716309,3.890625,3.839844,5.558594,...,2.259766,133.250000,-0.480713,-0.225464,0.540527,-0.634277,0.320068,165.250000,-8.695312,13.296875
4999987,500.0,4.320312,8,4.832031,4.437500,0.698730,0.738281,3.890625,3.632812,5.558594,...,2.078125,80.625000,-0.634277,-0.480713,0.694824,-0.343262,-0.122070,75.187500,-7.687500,11.640625
4999988,500.0,3.839844,8,4.820312,4.277344,0.716309,0.578613,3.839844,3.632812,5.558594,...,1.958984,56.593750,-0.343262,-0.634277,-0.225464,0.320068,-1.044922,46.468750,-7.160156,11.273438
4999989,500.0,3.632812,8,4.437500,3.931641,0.738281,0.458984,3.632812,3.388672,5.281250,...,1.906250,47.937500,0.320068,-0.343262,-0.480713,-0.122070,0.389160,37.812500,-6.835938,11.460938


In [13]:
# This feature is imported from NB 0.1 EDA
test['peak_detected'] = pd.read_csv('../data/interim/test_peak_detected_prominence1.csv')
train['peak_detected'] = pd.read_csv('../data/interim/train_peak_detected_prominence1.csv')

In [14]:
train.to_csv('../data/interim/train_FE_batchKal1.4.csv', index=True)
test.to_csv('../data/interim/test_FE_batchKal1.4.csv', index=True)